In [55]:
import pandas as pd
import sklearn.feature_extraction.text as tx, sklearn.naive_bayes as nb, sklearn.linear_model as lm, sklearn.ensemble as en, sklearn.metrics as me, sklearn.model_selection as ms
from warnings import filterwarnings as fw
fw('ignore')

все слова с учетом регистра<br>
все в одном регистре<br>
без стоп слов<br>
выделение специфичных для классов<br>
-- -- 
лог регрессия, случайный лес, байес

In [56]:
ord('Н')%3

0

In [57]:
def fitpredict(xtrain,ytrain,xtest,ytest):
    
    lr =  lm.LogisticRegression()
    rfc = en.RandomForestClassifier()
    cnb = nb.ComplementNB()

    lr.fit(xtrain,ytrain)
    rfc.fit(xtrain,ytrain)
    cnb.fit(xtrain,ytrain)

    pred_lr = lr.predict(xtest)
    pred_rfc = rfc.predict(xtest)
    pred_cnb = cnb.predict(xtest)
    
    print('Логистическая регрессия: \n', me.classification_report(ytest, pred_lr, digits=5),
        'Случайный лес: \n', me.classification_report(ytest, pred_rfc, digits=5),
        'Наивный Байес: \n', me.classification_report(ytest, pred_cnb, digits=5))
    
    return lr, rfc, cnb

In [58]:
corpus = pd.read_csv('SMSpam.csv')

In [59]:
cnt, cntS, cntH = 0, 0, 0
for i in corpus['Message']:
    cnt += len(i.split(' '))
for i in corpus[corpus['Category']=='spam']['Message']:
    cntS += len(i.split(' '))
for i in corpus[corpus['Category']=='ham']['Message']:
    cntH += len(i.split(' '))
print('Всего строк:', corpus.shape[0], '\nИз них', 
    corpus['Category'].value_counts()['spam'], 'spam и',corpus['Category'].value_counts()['ham'], 'ham', 
    '\nВсего слов: ', cnt, '\nиз них', cntS, 'spam и', cntH, 'ham')

Всего строк: 5572 
Из них 747 spam и 4825 ham 
Всего слов:  87265 
из них 17788 spam и 69477 ham


In [60]:
xtrain, xtest, ytrain, ytest = ms.train_test_split(corpus['Message'],corpus['Category'], test_size=0.2)
print('Обучающая выборка:', xtrain.shape,'\nТестовая выборка', xtest.shape)

Обучающая выборка: (4457,) 
Тестовая выборка (1115,)


-- -- 
Используем все слова с учетом регистра:

In [61]:
cv = tx.CountVectorizer(lowercase=False)
cv.fit(corpus['Message'])
print('Всего', len(cv.vocabulary_.keys()), 'токенов')

tr_xtrain = cv.transform(xtrain)
tr_xtest = cv.transform(xtest)

Всего 10857 токенов


In [62]:
fitpredict(tr_xtrain, ytrain, tr_xtest, ytest)

Логистическая регрессия: 
               precision    recall  f1-score   support

         ham    0.98381   0.99897   0.99133       973
        spam    0.99213   0.88732   0.93680       142

    accuracy                        0.98475      1115
   macro avg    0.98797   0.94315   0.96407      1115
weighted avg    0.98487   0.98475   0.98439      1115
 Случайный лес: 
               precision    recall  f1-score   support

         ham    0.98281   0.99897   0.99083       973
        spam    0.99206   0.88028   0.93284       142

    accuracy                        0.98386      1115
   macro avg    0.98744   0.93963   0.96183      1115
weighted avg    0.98399   0.98386   0.98344      1115
 Наивный Байес: 
               precision    recall  f1-score   support

         ham    0.99464   0.95375   0.97377       973
        spam    0.75275   0.96479   0.84568       142

    accuracy                        0.95516      1115
   macro avg    0.87369   0.95927   0.90972      1115
weighted avg 

(LogisticRegression(), RandomForestClassifier(), ComplementNB())

-- -- 
Используем все слова, но в одном регистре:

In [63]:
cv = tx.CountVectorizer()
cv.fit(corpus['Message'])
print('Всего', len(cv.vocabulary_.keys()), 'токенов')

tr_xtrain = cv.transform(xtrain)
tr_xtest = cv.transform(xtest)

Всего 8709 токенов


In [64]:
fitpredict(tr_xtrain, ytrain, tr_xtest, ytest)

Логистическая регрессия: 
               precision    recall  f1-score   support

         ham    0.98577   0.99692   0.99131       973
        spam    0.97710   0.90141   0.93773       142

    accuracy                        0.98475      1115
   macro avg    0.98144   0.94916   0.96452      1115
weighted avg    0.98467   0.98475   0.98449      1115
 Случайный лес: 
               precision    recall  f1-score   support

         ham    0.97887   1.00000   0.98932       973
        spam    1.00000   0.85211   0.92015       142

    accuracy                        0.98117      1115
   macro avg    0.98944   0.92606   0.95474      1115
weighted avg    0.98156   0.98117   0.98051      1115
 Наивный Байес: 
               precision    recall  f1-score   support

         ham    0.99577   0.96711   0.98123       973
        spam    0.81176   0.97183   0.88462       142

    accuracy                        0.96771      1115
   macro avg    0.90377   0.96947   0.93292      1115
weighted avg 

(LogisticRegression(), RandomForestClassifier(), ComplementNB())

-- -- 
Найдем все стоп-слова, которые встречаются в большом количестве сообщений (более 500 , что около 9% всей выборки) и уберем их:

In [65]:
cv = tx.CountVectorizer(max_df = 500)
cv.fit(corpus['Message'])
stop = cv.stop_words_
print('Всего', len(stop), 'токенов')
print(stop)

cv = tx.CountVectorizer(analyzer = 'word', stop_words = stop)
cv.fit(corpus['Message'])
tr_xtrain = cv.transform(xtrain)
tr_xtest = cv.transform(xtest)

Всего 15 токенов
{'to', 'me', 'that', 'is', 'of', 'for', 'have', 'it', 'and', 'the', 'your', 'you', 'call', 'my', 'in'}


In [66]:
fitpredict(tr_xtrain, ytrain, tr_xtest, ytest)

Логистическая регрессия: 
               precision    recall  f1-score   support

         ham    0.98580   0.99897   0.99234       973
        spam    0.99225   0.90141   0.94465       142

    accuracy                        0.98655      1115
   macro avg    0.98902   0.95019   0.96850      1115
weighted avg    0.98662   0.98655   0.98627      1115
 Случайный лес: 
               precision    recall  f1-score   support

         ham    0.97885   0.99897   0.98881       973
        spam    0.99180   0.85211   0.91667       142

    accuracy                        0.98027      1115
   macro avg    0.98533   0.92554   0.95274      1115
weighted avg    0.98050   0.98027   0.97962      1115
 Наивный Байес: 
               precision    recall  f1-score   support

         ham    0.99576   0.96506   0.98017       973
        spam    0.80233   0.97183   0.87898       142

    accuracy                        0.96592      1115
   macro avg    0.89904   0.96844   0.92957      1115
weighted avg 

(LogisticRegression(), RandomForestClassifier(), ComplementNB())

Найдем все стоп-слова, которые встречаются в маленьком количестве сообщений (менее 10, что около 0.18% всей выборки) и уберем:

In [67]:
cv = tx.CountVectorizer(min_df = 10)
cv.fit(corpus['Message'])
stop = cv.stop_words_
print('Всего', len(stop), 'токенов отмечены как стоп-слова.')
#print(stop) // не будем выводить список слов, так как он слишком большой

cv = tx.CountVectorizer(analyzer = 'word', stop_words = stop)
cv.fit(corpus['Message'])
tr_xtrain = cv.transform(xtrain)
tr_xtest = cv.transform(xtest)
print('Работаем с {0} - {1} = {2}'.format(len(cv.vocabulary_) + len(stop), len(stop), len(cv.vocabulary_)), 'токенами.')

Всего 7691 токенов отмечены как стоп-слова.
Работаем с 8709 - 7691 = 1018 токенами.


In [68]:
fitpredict(tr_xtrain, ytrain, tr_xtest, ytest)

Логистическая регрессия: 
               precision    recall  f1-score   support

         ham    0.98577   0.99692   0.99131       973
        spam    0.97710   0.90141   0.93773       142

    accuracy                        0.98475      1115
   macro avg    0.98144   0.94916   0.96452      1115
weighted avg    0.98467   0.98475   0.98449      1115
 Случайный лес: 
               precision    recall  f1-score   support

         ham    0.98278   0.99692   0.98980       973
        spam    0.97656   0.88028   0.92593       142

    accuracy                        0.98206      1115
   macro avg    0.97967   0.93860   0.95786      1115
weighted avg    0.98198   0.98206   0.98166      1115
 Наивный Байес: 
               precision    recall  f1-score   support

         ham    0.99265   0.97122   0.98182       973
        spam    0.82822   0.95070   0.88525       142

    accuracy                        0.96861      1115
   macro avg    0.91043   0.96096   0.93353      1115
weighted avg 

(LogisticRegression(), RandomForestClassifier(), ComplementNB())

Используем встроенный список стоп-слов:

In [69]:
cv = tx.CountVectorizer(stop_words = 'english')
cv.fit(corpus['Message'])
tr_xtrain = cv.transform(xtrain)
tr_xtest = cv.transform(xtest)
fitpredict(tr_xtrain, ytrain, tr_xtest, ytest)

Логистическая регрессия: 
               precision    recall  f1-score   support

         ham    0.98283   1.00000   0.99134       973
        spam    1.00000   0.88028   0.93633       142

    accuracy                        0.98475      1115
   macro avg    0.99141   0.94014   0.96383      1115
weighted avg    0.98502   0.98475   0.98433      1115
 Случайный лес: 
               precision    recall  f1-score   support

         ham    0.97883   0.99794   0.98830       973
        spam    0.98374   0.85211   0.91321       142

    accuracy                        0.97937      1115
   macro avg    0.98129   0.92503   0.95075      1115
weighted avg    0.97946   0.97937   0.97873      1115
 Наивный Байес: 
               precision    recall  f1-score   support

         ham    0.99678   0.95581   0.97587       973
        spam    0.76374   0.97887   0.85802       142

    accuracy                        0.95874      1115
   macro avg    0.88026   0.96734   0.91695      1115
weighted avg 

(LogisticRegression(), RandomForestClassifier(), ComplementNB())

-- -- 
Используем только слова с высокой важностью:

In [70]:
cv = tx.CountVectorizer()
cv.fit(corpus['Message'])
tr_xtrain = cv.transform(xtrain)
tr_xtest = cv.transform(xtest)

e = en.RandomForestClassifier()
e.fit(tr_xtrain,ytrain)
e.score(tr_xtest,ytest)

0.9838565022421525

In [71]:
imp = pd.DataFrame({'name' : cv.vocabulary_.keys(), 'importance' : e.feature_importances_}) 
imp = imp.sort_values(by = 'importance', ascending = False)
top100 = imp.head(n=1000)['name'].to_numpy()

cv = tx.CountVectorizer(vocabulary = top100)
cv.fit(corpus['Message'])
tr_xtrain = cv.transform(xtrain)
tr_xtest = cv.transform(xtest)

fitpredict(tr_xtrain, ytrain, tr_xtest, ytest)

Логистическая регрессия: 
               precision    recall  f1-score   support

         ham    0.95379   0.99692   0.97487       973
        spam    0.96939   0.66901   0.79167       142

    accuracy                        0.95516      1115
   macro avg    0.96159   0.83297   0.88327      1115
weighted avg    0.95577   0.95516   0.95154      1115
 Случайный лес: 
               precision    recall  f1-score   support

         ham    0.96414   0.99486   0.97926       973
        spam    0.95495   0.74648   0.83794       142

    accuracy                        0.96323      1115
   macro avg    0.95955   0.87067   0.90860      1115
weighted avg    0.96297   0.96323   0.96126      1115
 Наивный Байес: 
               precision    recall  f1-score   support

         ham    0.97204   0.89311   0.93091       973
        spam    0.52941   0.82394   0.64463       142

    accuracy                        0.88430      1115
   macro avg    0.75072   0.85853   0.78777      1115
weighted avg 

(LogisticRegression(), RandomForestClassifier(), ComplementNB())

-- -- 
Используем tf-idf векторизатор:

In [72]:
tfidfv = tx.TfidfVectorizer()
tfidfv.fit(corpus['Message'])
tr_xtrain = tfidfv.transform(xtrain)
tr_xtest = tfidfv.transform(xtest)

In [73]:
fitpredict(tr_xtrain, ytrain, tr_xtest, ytest)

Логистическая регрессия: 
               precision    recall  f1-score   support

         ham    0.96524   0.99897   0.98182       973
        spam    0.99074   0.75352   0.85600       142

    accuracy                        0.96771      1115
   macro avg    0.97799   0.87625   0.91891      1115
weighted avg    0.96849   0.96771   0.96579      1115
 Случайный лес: 
               precision    recall  f1-score   support

         ham    0.98184   1.00000   0.99084       973
        spam    1.00000   0.87324   0.93233       142

    accuracy                        0.98386      1115
   macro avg    0.99092   0.93662   0.96158      1115
weighted avg    0.98415   0.98386   0.98338      1115
 Наивный Байес: 
               precision    recall  f1-score   support

         ham    0.98956   0.97431   0.98187       973
        spam    0.84076   0.92958   0.88294       142

    accuracy                        0.96861      1115
   macro avg    0.91516   0.95194   0.93241      1115
weighted avg 

(LogisticRegression(), RandomForestClassifier(), ComplementNB())